In [33]:
path_img = '../../../opt/carnd_p3/data/IMG/'
path_data = '../../../opt/carnd_p3/data/driving_log.csv'
#path_img = 'IMG1/'
#path_data = 'driving_log.csv'

from keras.models import Sequential
from keras.layers import Lambda
from keras.layers import Dense, Flatten, Conv2D, Dropout, Cropping2D
from sklearn.model_selection import train_test_split
import sklearn.utils as utls
import imageio as imgio
import numpy as np
import cv2
import csv

samples = []
with open(path_data) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if line[0] == 'center':
            continue
        samples.append(line)


train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, path_image, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        utls.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = path_image+'/'+batch_sample[0].split('/')[-1]
                loaded_center_image = imgio.imread(name)
                center_image = loaded_center_image
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield utls.shuffle(X_train, y_train)

def image_manipulation(in_image):
    dim = (200, 66)
    resized_img = cv2.resize(in_image, dim, interpolation = cv2.INTER_AREA)
    image_yuv = cv2.cvtColor(resized_img, cv2.COLOR_RGB2YUV)
    #Y_img, U_img, V_img = cv2.split(image_yuv)
    #out_image = np.stack([Y_s, U_i, V_i])
    return image_yuv

batch_size = 32

#images, measurements = read_data(path_img, path_data)
# compile and train the model using the generator function
train_generator = generator(train_samples, path_img, batch_size=batch_size)
validation_generator = generator(validation_samples, path_img, batch_size=batch_size)

ch, row, col = 3, 160, 320
cropping_rows_top = 50
cropping_rows_bottom = 20
ch_cropped, row_cropped, col_cropped = ch, row-(cropping_rows_top+cropping_rows_bottom), col
ch_target, row_target, col_target = 3, 66, 200

## Create a CNN model. We are taking the Nvidia's End-to-End Deep Learning model
model = Sequential()
# Cropping
model.add(Cropping2D(cropping=((cropping_rows_top, cropping_rows_bottom), (0,0)), input_shape=(row,col,ch)))
# Normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(row_cropped,col_cropped,ch_cropped), output_shape=(row_cropped,col_cropped,ch_cropped)))
# Convolutional layer 1
model.add(Conv2D(24, (5,5), subsample=(2,2), activation = 'relu', padding='valid'))
# Convolutional layer 2
model.add(Conv2D(36, (5,5), subsample=(2,2), activation = 'relu', padding='valid'))
# Convolutional layer 3
model.add(Conv2D(48, (5,5), subsample=(2,2), activation = 'relu', padding='valid'))
# Convolutional layer 4
model.add(Conv2D(64, (3,3), activation = 'relu'))
# Convolutional layer 5
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Flatten())
# Dropout layer
model.add(Dropout(0.5))
# Fully connected layers
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, \
            steps_per_epoch=np.ceil(len(train_samples)/batch_size), \
            validation_data=validation_generator, \
            validation_steps=np.ceil(len(validation_samples)/batch_size), \
            epochs=8, verbose=1)

model.save('model.h5')

Epoch 1/5


FileNotFoundError: No such file: '/home/masterartem/Studies/UdacitySelfDriving/BehavioralCloning/BehavioralCloning/IMG1/center_2016_12_01_13_30_48_287.jpg'